https://huggingface.co/course/chapter7/6?fw=pt

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load PQuAD dataset

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("Gholamreza/pquad")

  0%|          | 0/3 [00:00<?, ?it/s]

## Extract unique contexts

In [ ]:
%%time
from tqdm.auto import tqdm
training_corpus = []

for i in tqdm(range(len(raw_datasets["train"]))):
  training_corpus.append(raw_datasets["train"][i]["context"])

  0%|          | 0/63994 [00:00<?, ?it/s]

CPU times: user 7.81 s, sys: 193 ms, total: 8.01 s
Wall time: 8.02 s


In [ ]:
%%time
print("Number of contexts:", len(training_corpus))
training_corpus = list(set(training_corpus))
print("Number of unique contexts:", len(training_corpus))
print()

Number of contexts: 63994
Number of unique contexts: 8968

CPU times: user 56.2 ms, sys: 0 ns, total: 56.2 ms
Wall time: 56.5 ms


## Tokenize the dataset

### Load Tokenizer

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "HooshvareLab/distilbert-fa-zwnj-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
len(training_corpus[1])

242

In [ ]:
%%time
lens_list = []
for i in range(1000):
  a = tokenizer(training_corpus[i])
  lens_list.append(len(a['input_ids']))

CPU times: user 728 ms, sys: 1.85 ms, total: 730 ms
Wall time: 730 ms


In [ ]:
max(lens_list)

370

In [ ]:
training_corpus[0]

'او سرانجام شاهنامه را در سال ۳۸۴ ه\u200d.ق، سه سال پیش از برتخت\u200cنشستنِ محمود، به\u200cپایان رساند. این ویرایش نخستینِ شاهنامه بود و فردوسی نزدیک به شانزده سالِ دیگر، در پرمایه\u200cتر و پیراسته کردن آن کوشید. این سال\u200cها هم\u200cزمان با برافتادن سامانیان و برآمدن سلطان محمود غزنوی بود. روشن است که در ویرایشِ نخستین، ستایش محمود نبود و به\u200cگمانْ ستایش منصور بن ابومنصور، بیش از آنچه که اکنون هست، بوده و چه\u200cبسا که ویرایشِ نخست به نام امیرک منصور بوده\u200cباشد. فردوسی در سال ۳۹۴ ه\u200d.ق در شصت و پنج سالگی بر آن شد که شاهنامه را به سلطان محمود پیشکش کند، و ازاین\u200cرو دست به کارِ تدوینِ ویرایشِ تازه\u200cای از شاهنامه شد. او در ویرایش دوم، بخش\u200cهای مربوط به پادشاهی ساسانیان را تکمیل کرد. پایان ویرایش دومِ شاهنامه در ۲۵ اسفند سال ۴۰۰ ه\u200d.ق برابر با ۸ مارس ۱۰۱۰ م[یادداشت 11] در هفتاد و یک سالگی فردوسی بوده\u200cاست:'

In [ ]:
context_length = 128

outputs = tokenizer(
    training_corpus[:2],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 3
Input chunk lengths: [128, 81, 72]
Chunk mapping: [0, 0, 1]


### Convert list of strings to HF Dataset

In [ ]:
def data_gen():
    for i in range(len(training_corpus)):
        yield {"content": training_corpus[i]}

In [ ]:
from datasets import Dataset
raw_dataset = Dataset.from_generator(data_gen)

In [ ]:
raw_dataset

Dataset({
    features: ['content'],
    num_rows: 8968
})

In [ ]:
raw_dataset[0]["content"]

'او سرانجام شاهنامه را در سال ۳۸۴ ه\u200d.ق، سه سال پیش از برتخت\u200cنشستنِ محمود، به\u200cپایان رساند. این ویرایش نخستینِ شاهنامه بود و فردوسی نزدیک به شانزده سالِ دیگر، در پرمایه\u200cتر و پیراسته کردن آن کوشید. این سال\u200cها هم\u200cزمان با برافتادن سامانیان و برآمدن سلطان محمود غزنوی بود. روشن است که در ویرایشِ نخستین، ستایش محمود نبود و به\u200cگمانْ ستایش منصور بن ابومنصور، بیش از آنچه که اکنون هست، بوده و چه\u200cبسا که ویرایشِ نخست به نام امیرک منصور بوده\u200cباشد. فردوسی در سال ۳۹۴ ه\u200d.ق در شصت و پنج سالگی بر آن شد که شاهنامه را به سلطان محمود پیشکش کند، و ازاین\u200cرو دست به کارِ تدوینِ ویرایشِ تازه\u200cای از شاهنامه شد. او در ویرایش دوم، بخش\u200cهای مربوط به پادشاهی ساسانیان را تکمیل کرد. پایان ویرایش دومِ شاهنامه در ۲۵ اسفند سال ۴۰۰ ه\u200d.ق برابر با ۸ مارس ۱۰۱۰ م[یادداشت 11] در هفتاد و یک سالگی فردوسی بوده\u200cاست:'

### Tokenize the dataset

In [ ]:
def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_dataset = raw_dataset.map(
    tokenize, batched=True, remove_columns=raw_dataset.column_names
)


  0%|          | 0/9 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 8055
})

**Smaller because sections smaller than 128 token have been discarded**



## Model

### Define a model with random weights

In [ ]:
from transformers import AutoTokenizer, DistilBertForMaskedLM, AutoConfig

config = AutoConfig.from_pretrained(
    model_checkpoint, # distillbert parsbert
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
model = DistilBertForMaskedLM(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"DistillBERT size: {model_size/1000**2:.1f}M parameters")

DistillBERT size: 75.8M parameters


### Collator

In [ ]:
from transformers import DataCollatorForLanguageModeling

# tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=True)

In [ ]:
"""ValueError: Asking to pad but the tokenizer does not have a padding token.
Please select a token to use as `pad_token` 
`(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a
new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`."""

tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # Fix
out = data_collator([tokenized_dataset[i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [ ]:
out

{'input_ids': tensor([[    2,     4,     4, 11408,     4,  1921,  1996,  6963,   622,     4,
           618,   590,  2237,  1996,  2076,     4, 14687,  1973,     9,     1,
          6186,   590,  1923,     9,  2805,  4639,   121,  1930,  5987,     1,
             4,  1976,   623,  9763,  2969,  1923, 15570,     1,     4,   590,
          1921, 40551,  3308,     9,  2013,   623,  3027, 12184,  2263,     1,
         19962,   121,  1930,     4,     9,     4,  1951,     9,  2247,  1927,
         26225,  3539,  3004,     4,   623,     1,  6742,  6186, 24294,  1976,
           121,  3565,  1933,  1932,  1921,     1,  3279,   590, 11175,  6186,
             4,   623,  1923,     9,     1, 11175, 11541,  2313, 16433,  1125,
         32816,   590,     4,  1925,     1,     4,  3096,  2065,   590,  2318,
           623,  2180,     9, 13967,  1932,     1,  2753,  1923,  2118,  5234,
          1127, 11541,  2318,     9,  2094,     4,  9763,  1921,     4,  8335,
          1195,   622,   121,     4,  

## Train

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir=f"{model_checkpoint}-MLM-pquad_2",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    gradient_accumulation_steps=8,
    num_train_epochs=20,
    weight_decay=0.1,
    warmup_steps=100,
    lr_scheduler_type="cosine",
    learning_rate=8e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/content/HooshvareLab/distilbert-fa-zwnj-base-MLM-pquad_2 is already a clone of https://huggingface.co/Gholamreza/distilbert-fa-zwnj-base-MLM-pquad_2. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [ ]:
trainer.train()


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8055
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 620
  Number of trainable parameters = 75812112


Step,Training Loss,Validation Loss
100,6.714000,6.435232
200,6.421600,6.134829
300,6.166700,5.923302
400,5.950200,5.690486
500,5.657800,5.283132
600,5.379000,5.174822


***** Running Evaluation *****
  Num examples = 8055
  Batch size = 32
***** Running Evaluation *****
  Num examples = 8055
  Batch size = 32
***** Running Evaluation *****
  Num examples = 8055
  Batch size = 32
***** Running Evaluation *****
  Num examples = 8055
  Batch size = 32
***** Running Evaluation *****
  Num examples = 8055
  Batch size = 32
***** Running Evaluation *****
  Num examples = 8055
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=620, training_loss=6.023483879335465, metrics={'train_runtime': 972.2659, 'train_samples_per_second': 165.695, 'train_steps_per_second': 0.638, 'total_flos': 5336375931629568.0, 'train_loss': 6.023483879335465, 'epoch': 19.98})

In [ ]:
trainer.push_to_hub()


Saving model checkpoint to HooshvareLab/distilbert-fa-zwnj-base-MLM-pquad_2
Configuration saved in HooshvareLab/distilbert-fa-zwnj-base-MLM-pquad_2/config.json
Model weights saved in HooshvareLab/distilbert-fa-zwnj-base-MLM-pquad_2/pytorch_model.bin
tokenizer config file saved in HooshvareLab/distilbert-fa-zwnj-base-MLM-pquad_2/tokenizer_config.json
Special tokens file saved in HooshvareLab/distilbert-fa-zwnj-base-MLM-pquad_2/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/289M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

Upload file runs/Feb20_12-42-12_8abe2dc88bd9/1676896934.9458582/events.out.tfevents.1676896934.8abe2dc88bd9.19…

Upload file runs/Feb20_12-37-41_8abe2dc88bd9/events.out.tfevents.1676896674.8abe2dc88bd9.190.0: 100%|#########…

Upload file runs/Feb20_12-42-12_8abe2dc88bd9/events.out.tfevents.1676896934.8abe2dc88bd9.190.2: 100%|#########…

Upload file runs/Feb20_12-37-41_8abe2dc88bd9/1676896674.6715443/events.out.tfevents.1676896674.8abe2dc88bd9.19…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/Gholamreza/distilbert-fa-zwnj-base-MLM-pquad_2
   3041722..6a3a1bf  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Gholamreza/distilbert-fa-zwnj-base-MLM-pquad_2
   3041722..6a3a1bf  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}, 'dataset': {'name': 'generator', 'type': 'generator', 'config': 'default', 'split': 'train', 'args': 'default'}}
To https://huggingface.co/Gholamreza/distilbert-fa-zwnj-base-MLM-pquad_2
   6a3a1bf..77fe293  main -> main

   6a3a1bf..77fe293  main -> main



'https://huggingface.co/Gholamreza/distilbert-fa-zwnj-base-MLM-pquad_2/commit/6a3a1bf6d570c6a2dcfb952c8a7396053b3e0b84'

## Test: Tokenizing english text

In [ ]:
tokenizer("سلام من خودم هستم")

In [ ]:
tokenizer("Hello I am myself")